# Compiling stages


The pipeline comprises two stages. The first stage is handled by the cython compiler, which transforms Cython source into optimized and platform-independent C or C++.
The second stage compiles the generated C or C++ source into a shared library with a standard C or C++ compiler. The resulting shared library is platform dependent. It is a shared object file with a *.so* extension on Linux or Mac OS X, and is a dynamic library
with a *.pyd* extension on Windows.

This pipeline can be run either automatically, without user involvement (making Cython feel much like Python), or explicitly by the end user when more control is required.


# Explicitly Compilation with distutils

```
~$ python setup.py build_ext --inplace
```

> The cython compiler is a source-to-source compiler, and the generated code is highly optimized. It is not uncommon for Cython generated C code to be faster than typical hand-written C. When the author teaches Cython, students often write C equivalents to Cython’s code; the Cython version is nearly always faster, and for equivalent algorithms is never slower. Cython’s generated C code is also highly portable, supporting all common C compilers and many Python versions simultaneously. 

*book "Cython: a guide for Python Programmers" pg. 13*

# Implicitly Compilation with Jupyter

Jupyter has several magic commands (click [here](https://notebook.community/acrispin/cython/docs/examples/Cython%20Magics) to check them) to allow dynamic compilation of Cython code.

In [6]:
%load_ext cython

The cython extension is already loaded. To reload it, use:
  %reload_ext cython


In [7]:
%%cython
def fib(int n):
    cdef int i
    cdef double a=0.0 
    cdef double b=1.0
    for i in range(n):
        a, b = a + b, a
    return a

In [8]:
fib(90)

2.880067194370816e+18

# Cython Typing

![](imgs/c_types.png)

Type declaration could be in the following way:

In [15]:
%%cython
cdef int i
cdef int N=2000
cdef float dx, s=0.0

or:

In [17]:
%%cython
cdef:
    int i, N=2000
    float dx, s=0.0

To enable type inference for a function, we can use the decorator form of infer_types:

In [20]:
%%cython
cimport cython

@cython.infer_types(True)
def more_inference():
    i = 1
    d = 2.0
    c = 3+4j
    r = i * d + c
    return r

## Poiters

Pointers can be declared in this way:

In [23]:
%%cython
cdef int *a, *b

Or this way:

In [25]:
%%cython
cdef int* a, b

Dereferencing pointers in Cython is different than in C. Because the Python language already uses the *args and **kwargs syntax to allow arbitrary positional and keyword arguments and to support function argument unpacking, Cython does not support the *a syntax to dereference a C pointer. Instead, we index into the pointer at location 0 to dereference a pointer in Cython. This syntax also works to dereference a pointer in C.

In [14]:
%%cython
cdef double golden_ratio
cdef double *p_double

p_double = &golden_ratio

p_double[0] = 1.618
print(golden_ratio)

1.618


## Statically Declaring Variables with a Python Type

Not all Python types can be statically declared: they must be implemented in C and
Cython must have access to the declaration. 

The built-in Python types already satisfy
these requirements, and declaring them is straightforward

In [26]:
%%cython
cdef list particles, modified_particles
cdef dict names_from_particles
cdef str pname
cdef set unique_particles

# C division vs Cython Division

C and Python have markedly different behavior when computing. Cython uses Python semantics by default.

To obtain C semantics, we can use the cdivision compiler directive, either at the global module level, or in a directive comment ```# cython: cdivision=True```:

In [32]:
%%cython
cimport cython

@cython.cdivision(True)
def divides(int a, int b):
    return a / b

In [29]:
divides(-1, 2)

0

In [31]:
divides(10, 2)

5

Or within a function with a context manager:

In [33]:
%%cython
cimport cython

def remainder(int a, int b):
    with cython.cdivision(True):
        return a % b

In [34]:
remainder(5, 3)

2